## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin


C:\Users\darap\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Loading the Data

In [2]:
ds = pd.read_csv("TB_ModelingdataV1.csv")
da = pd.read_csv("Yield_05-05-25.csv")
db = pd.read_csv("Yield_04-15-25.csv")

In [3]:
#Comment if you want to run only on Testbed Data
ds = ds[ds["Experiment"] != "SHTFG"]

In [4]:
ds  = ds.rename(columns = 
          {'ndvi_mean' : "NDVI_mean",
          'gndvi_mean' : "GNDVI_mean",
          'savi_mean' : "SAVI_mean",
          'msavi_mean' : "MSAVI_mean",
         'PT Height (mm)' : "MeanHeight(mm)",
         "Total Biomass (kg/ha)" : "Biomass(kg/ha)"})

In [5]:
db = db.rename(columns = 
        {
         'PT_Height(mm)' : "MeanHeight(mm)",
        "Total Biomass (kg/ha)" : "Biomass(kg/ha)"})

In [6]:
da = da.rename(columns = 
        {
         'PT_Height(mm)' : "MeanHeight(mm)" ,
        "Total Biomass (kg/ha)" : "Biomass(kg/ha)"})

In [7]:
# Concatenate only the matching columns (intersection)
df = pd.concat([da, db, ds], join='inner')

df


,Experiment,Date,Plot,Strip,MeanHeight(mm),NDVI_mean,GNDVI_mean,SAVI_mean,MSAVI_mean,Biomass(kg/ha)
0,TB,5/5/2025,14.1,14,227.620000,0.692000,0.632000,1.038000,0.818000,2605.290000
1,TB,5/5/2025,14.1,15,223.680000,0.700000,0.642000,1.050000,0.824000,2647.260000
2,TB,5/5/2025,14.1,16,176.820000,0.708000,0.652000,1.062000,0.829000,2511.700000
3,TB,5/5/2025,14.1,17,192.360000,0.708000,0.657000,1.062000,0.829000,3076.840000
4,TB,5/5/2025,14.1,18,209.770000,0.713000,0.653000,1.070000,0.833000,2815.820000
...,...,...,...,...,...,...,...,...,...,...
295,Testbed,2024-11-01,28.4,18,34.879460,0.444056,0.458791,0.666019,0.614980,1663.268656
296,Testbed,2024-11-01,28.4,19,40.842078,0.435176,0.460498,0.652702,0.606411,1356.823129
297,Testbed,2024-11-01,28.4,20,41.148287,0.435192,0.460140,0.652725,0.606426,1431.214846
298,Testbed,2024-11-01,28.4,21,44.628654,0.422257,0.458484,0.633324,0.593751,1476.081167


## Upgrading the Date Column

In [8]:
# Ensure the 'Date' column is in datetime format (handles mixed formats)
df['Date'] = pd.to_datetime(df['Date'], format='mixed')

# Calculate Julian date (day of the year)
df['JulianDate'] = df['Date'].dt.dayofyear
df

,Experiment,Date,Plot,Strip,MeanHeight(mm),NDVI_mean,GNDVI_mean,SAVI_mean,MSAVI_mean,Biomass(kg/ha),JulianDate
0,TB,2025-05-05,14.1,14,227.620000,0.692000,0.632000,1.038000,0.818000,2605.290000,125
1,TB,2025-05-05,14.1,15,223.680000,0.700000,0.642000,1.050000,0.824000,2647.260000,125
2,TB,2025-05-05,14.1,16,176.820000,0.708000,0.652000,1.062000,0.829000,2511.700000,125
3,TB,2025-05-05,14.1,17,192.360000,0.708000,0.657000,1.062000,0.829000,3076.840000,125
4,TB,2025-05-05,14.1,18,209.770000,0.713000,0.653000,1.070000,0.833000,2815.820000,125
...,...,...,...,...,...,...,...,...,...,...,...
295,Testbed,2024-11-01,28.4,18,34.879460,0.444056,0.458791,0.666019,0.614980,1663.268656,306
296,Testbed,2024-11-01,28.4,19,40.842078,0.435176,0.460498,0.652702,0.606411,1356.823129,306
297,Testbed,2024-11-01,28.4,20,41.148287,0.435192,0.460140,0.652725,0.606426,1431.214846,306
298,Testbed,2024-11-01,28.4,21,44.628654,0.422257,0.458484,0.633324,0.593751,1476.081167,306


In [12]:
# class RoundAndConvertFloatsToInt(BaseEstimator, TransformerMixin):
#     def __init__(self, decimals, exclude_cols=None):
#         self.decimals = decimals
#         self.exclude_cols = exclude_cols if exclude_cols is not None else []

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         X = X.copy()
#         float_cols = X.select_dtypes(include='float').columns
#         cols_to_transform = [col for col in float_cols if col not in self.exclude_cols]
#         X[cols_to_transform] = X[cols_to_transform].round(self.decimals).astype(float)

#         # Optional: print for verification
#         print("\nTransformed Features (excluding JulianDate):\n")
#         print(X.head(5))

#         return X

## Train, Validate and Test the Model

In [13]:
# Define the independent variables (features) and the target variable
features = ["MeanHeight(mm)","NDVI_mean", "GNDVI_mean", "SAVI_mean", "JulianDate"]
target = 'Biomass(kg/ha)'

# Ensure your data is clean and handle missing values
X = df[features]
y = df[target]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Preprocessing pipeline - Include all your preprocessing steps in below function to be applied to new data
preprocessor = Pipeline(steps=[
#     ('round_and_int', RoundAndConvertFloatsToInt(decimals=2, exclude_cols=["JulianDate"])),
    ('scaler', StandardScaler())
])

# have variable names as shown below _preprocessed
# Apply preprocessing
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Define Ridge regression and hyperparameter tuning
param_grid = {'alpha': [0.2]}  # Exploring different alpha values
ridge = Ridge()
# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(ridge, param_grid, cv=9, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train_preprocessed, y_train)

# Always register your model into variable named "model"
model = grid_search.best_estimator_

# Predict on test set
y_pred = model.predict(X_test_preprocessed)

# Evaluation Metrics (using absolute predictions)
test_mse = mean_squared_error(y_test, y_pred)
test_rmse = test_mse ** 0.5 
test_mae = mean_absolute_error(y_test, y_pred)  # MAE
test_r2 = r2_score(y_test, y_pred)  # R² Score

# Print evaluation results
print(f"Best Alpha: {grid_search.best_params_['alpha']}")
print(f"Test MSE: {test_mse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test R² Score: {test_r2:.4f}")



Transformed Features (excluding JulianDate):

     MeanHeight(mm)  NDVI_mean  GNDVI_mean  SAVI_mean  JulianDate
94           332.78       0.73        0.67       1.09         125
118           68.48       0.55        0.61       0.82         291
200          189.52       0.72        0.71       1.09         291
155          174.82       0.75        0.72       1.13         291
145          111.90       0.66        0.67       0.98         291

Transformed Features (excluding JulianDate):

     MeanHeight(mm)  NDVI_mean  GNDVI_mean  SAVI_mean  JulianDate
55           362.36       0.74        0.68       1.11         125
73           469.34       0.69        0.64       1.04         125
33           208.34       0.71        0.66       1.06         125
276           31.75       0.44        0.47       0.65         306
256           21.17       0.48        0.50       0.71         306
Best Alpha: 0.2
Test MSE: 255660.6551
Test RMSE: 505.6290
Test MAE: 360.8043
Test R² Score: 0.9246


In [11]:
# Print Regression Equation
coefficients = grid_search.best_estimator_.coef_
intercept = grid_search.best_estimator_.intercept_

# Assuming feature names are available (e.g., from a DataFrame)
# Otherwise, use X_train.shape[1] to construct generic variable names
try:
    feature_names = X_train.columns
except AttributeError:
    feature_names = [f"x{i+1}" for i in range(X_train.shape[1])]

# Construct the equation
equation = f"y = {intercept:.4f} " + " + ".join(
    [f"{coef:.4f}*{name}" for coef, name in zip(coefficients, feature_names)]
)

print("Regression Line Equation:")
print(equation)


Regression Line Equation:
y = 2926.5883 1606.5636*MeanHeight(mm) + -269.2503*NDVI_mean + 330.3815*GNDVI_mean + -286.1267*SAVI_mean + -168.5022*JulianDate
